# ModelSeed universal model
- 2022-04-14
- Minghao Gong

In [55]:
import requests
import json
import os
import pandas as pd

In [46]:
try:
    os.mkdir("../output/ModelSeed/")
except:
    None
try:
    os.mkdir("../input/ModelSeed/")
except:
    None

In [47]:
# r = requests.get("https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/Pathways/HopeScenarios.txt?raw=True")
# with open("../input/ModelSeed/HopeScenarios.txt", "w") as f:
#     f.write(r.text)

# r = requests.get("https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/Pathways/KEGG.pathways?raw=True")
# with open("../input/ModelSeed/KEGG.pathways", "w") as f:
#     f.write(r.text)

# r = requests.get("https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/Pathways/plantdefault.pathways.tsv?raw=True")
# with open("../input/ModelSeed/plantdefault.pathways.tsv", "w") as f:
#     f.write(r.text)

## pathway request

In [49]:
r = requests.get("https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/Pathways/ModelSEED_Subsystems.tsv?raw=True")
with open("../input/ModelSeed/ModelSEED_Subsystems.tsv", "w") as f:
    f.write(r.text)

## compounds request

In [51]:
r = requests.get("https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/compounds.tsv?raw=True")
with open("../input/ModelSeed/compounds.tsv", "w") as f:
    f.write(r.text)

In [61]:
r = requests.get("https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/compounds.json?raw=True")
with open("../input/ModelSeed/compounds.json", "w") as f:
    f.write(r.text)

## reactions request

In [53]:
r = requests.get("https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/reactions.tsv?raw=True")
with open("../input/ModelSeed/reactions.tsv", "w") as f:
    f.write(r.text)

In [76]:
r = requests.get("https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/reactions.json?raw=True")
with open("../input/ModelSeed/reactions.json", "w") as f:
    f.write(r.text)

In [62]:
cpd_df = pd.read_csv("../input/ModelSeed/compounds.tsv", sep = "\t") # using json formatted compound table for parsing
rxn_df = pd.read_csv("../input/ModelSeed/reactions.tsv", sep = '\t')
pwy_df = pd.read_csv("../input/ModelSeed/ModelSEED_Subsystems.tsv", sep = '\t')

In [67]:
cpd_df.shape # matched with the json format! So using json is fine!

(33992, 21)

In [77]:
rxn_df.shape # matched with the json format! So using json is fine!

(43774, 22)

In [64]:
with open("../input/ModelSeed/compounds.json", "r") as f:
    cpd_list = json.load(f)

In [78]:
with open("../input/ModelSeed/reactions.json", "r") as f:
    rxn_list = json.load(f)

In [79]:
len(rxn_list)

43774

----

In [161]:
import sys
from metDataModel.core import Compound, Reaction, Pathway, MetabolicModel
sys.path.append("/Users/gongm/Documents/projects/mass2chem/")
sys.path.append("/Users/gongm/Documents/projects/JMS/JMS/jms/")
import mass2chem
from jms import formula

# Porting compounds

In [158]:
cpd.__dict__

{'internal_id': 'cpd37302',
 'id': 'cpd37302',
 'name': '2-methylene-alpha-oxocyclopropanepropanoic acid; 3-(2-methylidenecyclopropyl)-2-oxopropanoic acid; beta-methylenecyclopropyl pyruvate; ketohypoglycin; methylenecyclopropylpyruvate',
 'db_ids': [('BrachyCyc', 'CPD-9739'),
  ('MaizeCyc', 'CPD-9739'),
  ('MetaCyc', 'CPD-9739'),
  ('PlantCyc', 'CPD-9739'),
  ('inchikey', 'IKTGVEIOCJLOFU-UHFFFAOYSA-M')],
 'neutral_formula': '',
 'neutral_mono_mass': 0.0,
 'charge': -1,
 'charged_formula': 'C7H7O3',
 'SMILES': 'C=C1CC1CC(=O)C(=O)[O-]',
 'inchi': '',
 'abbreviation': '2-methylene-alpha-oxocyclopropanepropanoic acid'}

In [172]:
myCpdlist = []
for seedCpd in cpd_list:
    cpd = Compound()
    cpd.abbreviation = seedCpd['abbreviation']
    cpd.id = seedCpd['id']
    cpd.internal_id = cpd.id
    cpd.db_ids = []
    try:
        for item in seedCpd['aliases']:
            record = tuple(item.split(": "))
            if record[0] == "Name":
                cpd.name = record[1]
            else:
                cpd.db_ids.append(record)
    except:
        None
    cpd.db_ids.append(('inchikey',seedCpd['inchikey']))
    cpd.charge = seedCpd['charge']
    cpd.charged_formula = seedCpd['formula']
    cpd.SMILES = seedCpd['smiles']
    
    try:
        cpd.neutral_formula = formula.adjust_charge_in_formula(cpd.charged_formula,cpd.charge)
    except:
        cpd.neutral_formula = ''
    try:
        cpd.neutral_mono_mass = round(mass2chem.formula.calculate_formula_mass(cpd.neutral_formula),6)
    except:
        cpd.neutral_mono_mass = 0
    myCpdlist.append(cpd)

In [173]:
len(myCpdlist)

33992

In [177]:
[x.__dict__ for x in myCpdlist[:2]]

[{'internal_id': 'cpd00001',
  'id': 'cpd00001',
  'name': 'H20; H2O; H3O+; HO-; Hydroxide ion; OH; OH-; Water; hydrogen oxide; hydroxide; hydroxide ion; hydroxyl; hydroxyl ion; oxonium; water',
  'db_ids': [('AraCyc', 'OH; WATER'),
   ('BiGG', 'h2o; oh1'),
   ('BrachyCyc', 'WATER'),
   ('KEGG', 'C00001; C01328'),
   ('MetaCyc', 'OH; OXONIUM; WATER'),
   ('inchikey', 'XLYOFNOQVPJJNP-UHFFFAOYSA-N')],
  'neutral_formula': 'H2O',
  'neutral_mono_mass': 18.010565,
  'charge': 0,
  'charged_formula': 'H2O',
  'SMILES': 'O',
  'inchi': '',
  'abbreviation': 'h2o'},
 {'internal_id': 'cpd00002',
  'id': 'cpd00002',
  'name': "ATP; Adenosine 5'-triphosphate; adenosine-5'-triphosphate; adenosine-triphosphate; adenylpyrophosphate",
  'db_ids': [('AraCyc', 'ATP'),
   ('BiGG', 'atp'),
   ('BrachyCyc', 'ATP'),
   ('KEGG', 'C00002'),
   ('MetaCyc', 'ATP'),
   ('inchikey', 'ZKHQWZAMYRWXGA-KQYNXXCUSA-K')],
  'neutral_formula': 'C10H16N5O13P3',
  'neutral_mono_mass': 506.995747,
  'charge': -3,
  'charg

# Porting reactions

In [75]:
rxn = Reaction()
rxn.__dict__

{'azimuth_id': '',
 'id': '',
 'source': [],
 'version': '',
 'status': '',
 'reactants': [],
 'products': [],
 'enzymes': [],
 'genes': [],
 'pathways': [],
 'ontologies': [],
 'species': '',
 'compartments': [],
 'cell_types': [],
 'tissues': []}

In [102]:
rxn_list[0]

{'abbreviation': 'R00004',
 'abstract_reaction': None,
 'aliases': ['AraCyc: INORGPYROPHOSPHAT-RXN',
  'BiGG: IPP1; PPA; PPA_1; PPAm',
  'BrachyCyc: INORGPYROPHOSPHAT-RXN',
  'KEGG: R00004',
  'MetaCyc: INORGPYROPHOSPHAT-RXN',
  'Name: Diphosphate phosphohydrolase; Inorganic diphosphatase; Inorganic pyrophosphatase; Pyrophosphate phosphohydrolase; diphosphate phosphohydrolase; inorganic diphosphatase; inorganic diphosphatase (one proton translocation); inorganicdiphosphatase; pyrophosphate phosphohydrolase'],
 'code': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0]',
 'compound_ids': 'cpd00001;cpd00009;cpd00012;cpd00067',
 'definition': '(1) H2O[0] + (1) PPi[0] <=> (2) Phosphate[0] + (1) H+[0]',
 'deltag': -3.46,
 'deltagerr': 0.05,
 'direction': '=',
 'ec_numbers': ['3.6.1.1'],
 'equation': '(1) cpd00001[0] + (1) cpd00012[0] <=> (2) cpd00009[0] + (1) cpd00067[0]',
 'id': 'rxn00001',
 'is_obsolete': 0,
 'is_transport': 0,
 'linked_reaction': 'rxn27946;rxn27947;rxn27948;rxn32487;

In [101]:
rxn_list[0].keys()

dict_keys(['abbreviation', 'abstract_reaction', 'aliases', 'code', 'compound_ids', 'definition', 'deltag', 'deltagerr', 'direction', 'ec_numbers', 'equation', 'id', 'is_obsolete', 'is_transport', 'linked_reaction', 'name', 'notes', 'pathways', 'reversibility', 'source', 'status', 'stoichiometry'])

In [89]:
test = rxn_list[10]['code']

In [91]:
rxn_list[10]['code']

'(2) cpd00076[0] <=> (1) cpd00027[0] + (1) cpd02298[0]'

In [94]:
import re
def extract_compounds(equation_code):
    [reactant_str, product_str] = equation_code.split("=")
    reactants = re.findall(r'\) (cpd\d+)\[',reactant_str)
    products = re.findall(r'\) (cpd\d+)\[',product_str)
    return(reactants,products)

reactants, products = extract_compounds(rxn_list[10]['code'])

In [110]:
def port_reaction(R):
    new = Reaction()
    new.id = R['id']
    reactants, products = extract_compounds(R['code'])
    new.reactants = reactants 
    new.products = products
    if R['ec_numbers']:
        if "|" in R['ec_numbers']:
            new.enzymes = ecs.split("|")
        else:
            new.enzymes = R['ec_numbers']       # this version of human-GEM may have it as string
    return new


In [115]:
myRxnlist = []
for R in rxn_list: 
    if R['is_transport']==0: # remove the transportation reactions
        myRxnlist.append(port_reaction(R))

In [116]:
print(f'There are {len(myRxnlist)} true reactions without considering transport in {len(rxn_list)} reactions')

There are 38269 true reactions without considering transport in 43774 reactions


----

# Port pathways

In [118]:
pwy = Pathway()
pwy.__dict__

{'azimuth_id': '',
 'id': '',
 'name': '',
 'source': [],
 'list_of_reactions': [],
 'status': ''}

In [151]:
subsys2reactions_dict = pwy_df.groupby('Sub-class')['Reaction'].apply(list).to_dict()

In [152]:
def generateMSPwy(number):
    res = "MSPwy" + str(number).zfill(4)
    return(res)

In [153]:
path_test = Pathway();path_test.__dict__

{'azimuth_id': '',
 'id': '',
 'name': '',
 'source': [],
 'list_of_reactions': [],
 'status': ''}

In [156]:
mysubsyslist = []
i = 1
for k,v in subsys2reactions_dict.items():
    new = Pathway()
    new.id = generateMSPwy(i)
    new.name = k
    new.source = "ModelSeed"
    new.list_of_reactions = v
    new.status: "testing"

    i +=1
    mysubsyslist.append(new)

In [157]:
len(mysubsyslist)

143

# Collect data and output 

In [178]:
note = """ModelSeed Universal Model. Add all reactions and compounds, """

## metabolicModel to export
MM = MetabolicModel()
MM.id = '' #
MM.meta_data = {
            'species': 'universal',
            'version': '',
            'sources': ['https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/, retrieved 2022-04-13'], #
            'status': '',
            'last_update': '20220413',  #
            'note': note,
        }
MM.list_of_pathways = [P.serialize() for P in mysubsyslist]
MM.list_of_reactions = [R.serialize() for R in  myRxnlist]
MM.list_of_compounds = [C.serialize() for C in myCpdlist]

In [179]:
# check output
[
MM.list_of_pathways[2],
MM.list_of_reactions[:2],
MM.list_of_compounds[100:102],
]

[{'id': 'MSPwy0003',
  'name': 'A cluster perhaps related to purine metabolism',
  'list_of_reactions': ['rxn00800', 'rxn03136', 'rxn13784']},
 [{'id': 'rxn00001',
   'reactants': ['cpd00001', 'cpd00012'],
   'products': ['cpd00009'],
   'genes': [],
   'enzymes': ['3.6.1.1']},
  {'id': 'rxn00002',
   'reactants': ['cpd00001', 'cpd00742'],
   'products': ['cpd00011', 'cpd00013'],
   'genes': [],
   'enzymes': ['3.5.1.54']}],
 [{'id': 'cpd00101',
   'name': "CPD-15317; D-Ribose 5-phosphate; D-ribose 5'-phosphate; D-ribose 5-phosphate; D-ribose-5-P; D-ribose-5-phosphate; D-ribose-5-phosphoric acid; RIBOSE-5P; Ribose 5-phosphate; aldehydo-D-ribose 5-phosphate; alpha-D-Ribose5-phosphate; alpha-D-ribose-5-phosphate; keto-D-ribose 5-phosphate; ribose-5-P; ribose-5-phosphate; ribose-5-phosphoric acid; ribose-5P",
   'identifiers': [('AraCyc', 'RIBOSE-5P'),
    ('BiGG', 'r5p'),
    ('BrachyCyc', 'RIBOSE-5P'),
    ('KEGG', 'C00117'),
    ('MetaCyc', 'CPD-15317; CPD-15895; RIBOSE-5P'),
    ('inc

In [181]:
with open('../output/ModelSeed/Universal_ModelSeed.json', "w") as f:
    json.dump(MM.serialize(),f)

In [182]:
with open('../output/ModelSeed/Universal_ModelSeed.json', "r") as f:
    test = json.load(f)